In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pyskip

In [ ]:
%%time
D = 1024
R = D // 4
disc  = pyskip.Tensor((D, D), 0)
for y in range(D):
    discriminant = R**2 - (y - D // 2 + 0.5)**2
    if discriminant < 0:
        continue
    x_0 = int(D // 2 - 0.5 - discriminant**0.5)
    x_1 = int(D // 2 - 0.5 + discriminant**0.5)  
    disc[x_0:x_1, y] = 1

In [ ]:
plt.imshow(disc.to_numpy())

In [ ]:
%%timeit
conv_disc = disc[D // 4:-D // 4, D // 4:-D // 4].eval()

In [ ]:
plt.imshow(disc[D // 4:-D // 4, D // 4:-D // 4].to_numpy())

In [ ]:
def conv_2d(tensor, kernel, padding=0, fill=0):
    pad_shape = (
      tensor.shape[0] + 2 * padding,
      tensor.shape[1] + 2 * padding,
    )
    pad = pyskip.Tensor(shape=pad_shape, dtype=tensor.dtype, val=fill)
    pad[padding:-padding, padding:-padding] = tensor
    
    out_shape = (
      pad_shape[0] - kernel.shape[0] + 1,
      pad_shape[1] - kernel.shape[1] + 1,
    )
    out = pyskip.Tensor(shape=out_shape, dtype=tensor.dtype, val=0)
    for y in range(kernel.shape[1]):
        for x in range(kernel.shape[0]):
            stop_x = pad_shape[0] - kernel.shape[0] + x + 1
            stop_y = pad_shape[1] - kernel.shape[1] + y + 1
            out += kernel[x, y] * pad[x:stop_x, y:stop_y]
    return out

In [ ]:
%%time 

edges = conv_2d(
    tensor=disc,
    kernel = pyskip.Tensor.from_list([
        [-1, 0, 1],
        [-2, 0, 2],
        [-1, 0, 1],
    ]),
    padding=1,
).eval()

In [ ]:
plt.imshow(np.abs(edges.to_numpy()))

In [ ]:
from scipy.signal import convolve2d

In [ ]:
disc_np = disc.to_numpy()

In [ ]:
%%time 

edges_np = convolve2d(
    disc_np, 
    np.array([
        [-1, 0, 1],
        [-2, 0, 2],
        [-1, 0, 1],
    ]),
)

In [ ]:
plt.imshow(np.abs(edges_np))

In [ ]:
def gaussian_kernel(size=3, std=1.0):
    """Returns a 2D Gaussian kernel array."""
    K = signal.gaussian(size, std=std).reshape(size, 1)
    return np.outer(K, K)